# Prep

In [1]:
import musicbrainzngs

# Authentication

In [5]:
musicbrainzngs.set_useragent(app="Opus Metadata Fetcher",version = "1.0")

In [6]:
artist_id = "c5c2ea1c-4bde-4f4d-bd0b-47b200bf99d6"
try:
    result = musicbrainzngs.get_artist_by_id(artist_id)
except WebServiceError as exc:
    print("Something went wrong with the request: %s" % exc)
else:
    artist = result["artist"]
    print("name:\t\t%s" % artist["name"])
    print("sort name:\t%s" % artist["sort-name"])

name:		The xx
sort name:	xx, The


In [7]:
result = musicbrainzngs.get_artist_by_id(artist_id,
              includes=["release-groups"], release_type=["album", "ep"])
for release_group in result["artist"]["release-group-list"]:
    print("{title} ({type})".format(title=release_group["title"],
                                    type=release_group["type"]))

xx (Album)
Coexist (Album)
I See You (Album)
Remixes (Compilation)
FACT Mix 70: The xx (Compilation)
Islands (EP)
Tour Only EP (EP)
iTunes Live From SoHo (EP)
iTunes Festival: London 2010 (EP)
Innervisions Remixes (EP)
Hivern Remixes (EP)
Live at Club 69 (Live)
KEXP Session (EP)


In [59]:
musicbrainzngs.get_recordings_by_isrc('US23S9528112', includes=['artists',
                                                                'releases',
                                                                'isrcs'], release_status=[], release_type=[])
# artists, 
# releases, 
# isrcs


{'isrc': {'id': 'US23S9528112',
  'recording-list': [{'id': '704ac8b9-5ee6-4c36-ab5d-5a72332b52e7',
    'title': '1st of tha Month',
    'length': '314466',
    'disambiguation': 'explicit, with intro',
    'artist-credit': [{'artist': {'id': '2fa45bbb-0efb-4950-9d40-94bf23cbfec1',
       'type': 'Group',
       'name': 'Bone Thugs‐n‐Harmony',
       'sort-name': 'Bone Thugs‐n‐Harmony'}}],
    'release-list': [{'id': '67d40d9f-2c89-4d49-8401-794e34f3e2c4',
      'title': 'E. 1999 Eternal',
      'status': 'Official',
      'quality': 'normal',
      'text-representation': {'language': 'eng', 'script': 'Latn'},
      'date': '1995-07-25',
      'country': 'US',
      'release-event-list': [{'date': '1995-07-25',
        'area': {'id': '489ce91b-6658-3307-9877-795b68554c98',
         'name': 'United States',
         'sort-name': 'United States',
         'iso-3166-1-code-list': ['US']}}],
      'release-event-count': 1,
      'barcode': '088561553920'},
     {'id': 'a4579d95-44bc-3de4-8

In [ ]:
# to get from isrc to label:
# isrc --> recording
# recording --> releases

In [74]:
test = musicbrainzngs.get_recording_by_id('704ac8b9-5ee6-4c36-ab5d-5a72332b52e7', includes=['releases'], release_status=[], release_type=[])
rls_ids = [rls['id'] for rls in test['recording']['release-list']]

In [75]:
rls_ids

['67d40d9f-2c89-4d49-8401-794e34f3e2c4',
 'a4579d95-44bc-3de4-8782-57b3872c4bcf',
 'cd4d69ae-98a0-4e5b-b512-722f71254a58',
 '98b5846e-4041-39a7-b200-846f7516fafd',
 '5e1806ad-6f0c-4894-b88f-486ee5717afb',
 'a3ceef00-c56e-4bc4-9e67-a390c2d1a91a',
 '7fa58a0e-d724-440b-9e98-6cbd760c5b32',
 '7047a0b0-15ec-4e1a-9028-f0618f65f7cb',
 'fcf8ccf7-c111-4a4a-b7a0-ba3a3d6d389e',
 'b71bfa10-25b0-457a-8b21-54e5e8e822ea',
 '5852bacd-8636-4683-9182-16fc19eebf5d',
 '099443c7-2578-46e7-8729-25b66aeeeae0',
 'd89f94fa-9a2b-4eb9-ae2a-53e0806eec55']

In [102]:
test2 = musicbrainzngs.get_release_by_id(id = rls_ids[1],includes='labels')
# print(test2)
print(test2['release']['label-info-count'])
print(test2['release']['label-info-list'][0]['label']['name'])


IndexError: list index out of range

In [42]:
record_search = musicbrainzngs.search_recordings(query='', limit=None, offset=None, strict=False, isrc = 'US23S9528112')
record_search['recording-list'][0]['release-list'][0]

{'id': '099443c7-2578-46e7-8729-25b66aeeeae0',
 'title': 'E. 1999 Eternal',
 'status': 'Official',
 'disambiguation': 'VMP Essentials',
 'artist-credit': [{'name': 'Bone Thugs‐n‐Harmony',
   'artist': {'id': '2fa45bbb-0efb-4950-9d40-94bf23cbfec1',
    'name': 'Bone Thugs‐n‐Harmony',
    'sort-name': 'Bone Thugs‐n‐Harmony'}}],
 'release-group': {'id': '5262e658-652f-33aa-af8f-6af28c2da7aa',
  'type': 'Album',
  'title': 'E. 1999 Eternal',
  'primary-type': 'Album'},
 'date': '2023-08',
 'country': 'US',
 'release-event-list': [{'date': '2023-08',
   'area': {'id': '489ce91b-6658-3307-9877-795b68554c98',
    'name': 'United States',
    'sort-name': 'United States',
    'iso-3166-1-code-list': ['US']}}],
 'medium-list': [{'position': '2',
   'format': '12" Vinyl',
   'track-list': [{'id': 'b5816721-f492-42ea-a01d-bf9c0fbfe028',
     'number': 'C4',
     'title': '1st of tha Month'}],
   'track-count': 9}],
 'medium-track-count': 17,
 'medium-count': 2,
 'artist-credit-phrase': 'Bone Thug

In [44]:
# ruthless attack muzick record label
musicbrainzngs.search_labels(query='', limit=None, offset=None, strict=False, ipi = '00128442384')

{'label-list': [{'id': '90594e37-b9cf-4f80-9960-2f8e28c3b34d',
   'type': 'Publisher',
   'ext:score': '100',
   'name': 'Ruthless Attack Muzick',
   'sort-name': 'Ruthless Attack Muzick',
   'ipi-list': ['00128442384'],
   'country': 'US',
   'area': {'id': '489ce91b-6658-3307-9877-795b68554c98',
    'type': 'Country',
    'name': 'United States',
    'sort-name': 'United States',
    'life-span': {'ended': 'false'}},
   'life-span': {'ended': 'false'}}],
 'label-count': 1}